- 案件：[リサイクル品を扱うネットショップのリサーチと商品登録業務をお手伝い頂けませんか？](https://crowdworks.jp/proposals/193638313#scroll_to_message)
- Notionページ：https://www.notion.so/5364556d13fe44d6afd582e0a49fb29a
- staff2@polyphony.tokyo
- Muzaiko20230606

# 動作検証

## ブラウザを起動しAmazonページを開く

In [594]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import datetime
import time
import re # 正規表現用
import pandas as pd

#ブラウザの設定
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
# 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# Chromeを起動するためのオプションにプロファイルを追加する
chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

#ブラウザの起動する
browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)
browser.implicitly_wait(1)

# Amazon商品ページを開く
url = 'https://www.amazon.co.jp/s?i=kitchen&bbn=3901642051&rh=n%3A3901642051&dc&fs=true&ds=v1%3AToEcktBQXhf5eAM6xUbKHgRKlL92dOwOpGKkc6hn4VI&qid=1686568956&ref=sr_ex_n_1' # 固定入力の場合
# print('▼ 調査対象の Amazon URL を入力してください') # 標準入力の場合
# url = input() # 標準入力の場合
browser.get(url)

# Amazon のウィンドウハンドラの保持　→　なくても大丈夫っぽい
amazon_window = browser.current_window_handle
# print(amazon_window)

# 情報格納用リスト
data_list = []

C:\Users\amuza\AppData\Local\Temp\ipykernel_2840\226476478.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)


In [ ]:
# 【動作テスト用】
# ページを直接開く
browser.get('')

# 大元（Amazon）のウィンドウハンドラの保持
amazon_window = browser.current_window_handle
print(amazon_window)

## Amazonページから情報を取得する

In [595]:
# Amazon商品＋ヤフオク商品群の基点要素を取得する
amazon_yafuoku_item_list = browser.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
# print(item_list)

# 親要素内の情報を取得する
for elem in amazon_yafuoku_item_list:
    
    # Amazon商品名
    amazon_name = elem.find_element(By.XPATH, './/h2//span').text
    print('Amazon商品名：', amazon_name)
    
    # Amazon購入実績
    try:
        amazon_sold_num = elem.find_element(By.XPATH, './/span[contains(text(), "購入されました")]').text
        print(amazon_sold_num)
    except:
        # print('▲ Amazon購入実績情報なし ▲')
        pass

    # Amazon価格
    try:
        amazon_price = int(elem.find_element(By.XPATH, './/span[@class="a-price-whole"]').text.replace(',', ''))
        # print('Amazon価格：', amazon_price, '円')
    except:
        print('▲ Amazon商品価格なし ▲')

    print('　▽')

    # Amazon商品URL
    amazon_url = elem.find_element(By.XPATH, './/h2/a').get_attribute('href')
    # print('Amazon商品URL：', amazon_url)
    
    # ヤフオク商品群の基点要素を取得する
    yafuoku_item_list = elem.find_elements(By.XPATH, './/table[@class="table vmd ytable"]')
    
    for elem in yafuoku_item_list:
    
        # ヤフオク商品名
        try:
            yafuoku_name = elem.find_element(By.XPATH, './/a[@class="__ylink"]').text
            print('ヤフオク商品名：', yafuoku_name)
        except:
            print('▲ 該当ヤフオク商品なし ▲')
        
        # ヤフオク現在価格
        try:
            yafuoku_price_now = int(elem.find_element(By.XPATH, './/div[contains(text(), "現在")]').text.replace('現在 : ￥', ''))
            # print('ヤフオク現在価格：', yafuoku_price_now, '円')
        except:
            print('▲ ヤフオク現在価格なし ▲')

        # # ヤフオク即決価格
        # try:
        #     yafuoku_price_final = int(elem.find_element(By.XPATH, './/div[contains(text(), "即決")]').text.replace('即決 : ￥', ''))
        #     # print(yafuoku_price_final)
        # except:
        #     print('▲ ヤフオク即決価格なし ▲')

        # # ヤフオク残り時間
        # try:
        #     yafuoku_item_left = elem.find_element(By.XPATH, './/div[contains(text(), "残り")]').text
        #     print(yafuoku_item_left)
        # except:
        #     print('▲ ヤフオク残り時間情報なし ▲')

        price_delta_now = amazon_price - yafuoku_price_now
        # print('A-Y参考現在価格差：', price_delta_now, '円')

        # price_delta_final = amazon_price - yafuoku_final_price
        # print('A-Y即決価格差：', price_delta_final)

        profit_rate_now = price_delta_now / yafuoku_price_now
        # print('参考現在利益率：', round(profit_rate_now * 100, 1), '%')

        # 利益率が 5% 以上なら
        if profit_rate_now > 0.1:
            # ヤフオク商品ページを開く
            try:
                elem.find_element(By.XPATH, './/a[@class="__ylink"]').click()
                print('=== ヤフオク商品ページを開きます ===')
            except:
                print('▲ ヤフオク商品ページを開けませんでした ▲')                
        else:
            print('▲ 利益を見込めないためヤフオク商品ページを開きませんでした ▲')

        print('')

    print('-')
    break

Amazon商品名： SALONIA サロニア | スピーディーイオン ドライヤー 【ブラック】 ヘアドライヤー 大風量 速乾 軽量 マイナスイオン 折り畳み式 冷温3段階調整
過去1か月で5000点以上購入されました
Amazon価格： 5373 円
　▽
ヤフオク商品名： SALONIA◆ドライヤー・ヘアアイロン スピーディーイオンドライヤー SL-013BK [BLACK]
ヤフオク現在価格： 5390 円
A-Y参考現在価格差： -17 円
参考現在利益率： -0.3 %
▲ 利益基準未達のためヤフオク商品ページを開きませんでした ▲

ヤフオク商品名： 【1円】SALONIA ドライヤー SL-013BK サロニア【訳あり品】
ヤフオク現在価格： 1 円
A-Y参考現在価格差： 5372 円
参考現在利益率： 537200.0 %
=== ヤフオク商品ページを開きます ===

ヤフオク商品名： SALONIA◆SALONIA/ドライヤー・ヘアアイロン スピーディーイオンドライヤー SL-013BK [BLACK]
ヤフオク現在価格： 4290 円
A-Y参考現在価格差： 1083 円
参考現在利益率： 25.2 %
=== ヤフオク商品ページを開きます ===

ヤフオク商品名： SALONIA◆ドライヤー・ヘアアイロン スピーディーイオンドライヤー SL-013BK [BLACK]
ヤフオク現在価格： 4290 円
A-Y参考現在価格差： 1083 円
参考現在利益率： 25.2 %
=== ヤフオク商品ページを開きます ===

ヤフオク商品名： サロニア スピーディーイオンドライヤー SL-013BK ブラック【PSEマークあり】【訳あり※折りまげ部分ゆるみ有】58 00064
ヤフオク現在価格： 4000 円
A-Y参考現在価格差： 1373 円
参考現在利益率： 34.3 %
=== ヤフオク商品ページを開きます ===

ヤフオク商品名： ★sr0567　SALONIA　スピーディーイオンドライヤー　SL-013BK　ブラック　温風3モード　サロニア　マイナスイオン　速乾　家電製品★
ヤフオク現在価格： 2800 円
A-Y参考現在価格差： 2573 円
参考現在利益率： 91.9 %
=== ヤフオク商品ページを開きます ===

ヤフオク

## ヤフオクページから情報を取得する

In [623]:
# 開いているすべてのウィンドウハンドラを取得（リスト形式）し、最後のウィンドウをアクティブにする
handle_array = browser.window_handles
print(handle_array)
browser.switch_to.window(handle_array[-1])

# ヤフオク商品ページのウィンドウハンドラの保持　→　不要っぽい
yafuoku_window = browser.current_window_handle
print(yafuoku_window)
print('最新のヤフオク商品ページをアクティブにしました')

['8856A3EB0B23D6AFF994EFC79AA31CFD', 'CE9D124FA3764CB56C8A3698B7BCFD05', 'EA1F127D66F592F0B46F6C2191EB1267', '60BF52C790BC88F8073C99A8B330ECFC', 'CDF4018A964461975D3EF2B92D0703F3', 'B846C23B532D298376F5C82664DD7442', 'B274FA4FEFB2CCAC4E8511B299CE59B6']
B274FA4FEFB2CCAC4E8511B299CE59B6
最新のヤフオク商品ページをアクティブにしました


In [ ]:
# 【動作テスト用】
# Amazonページから続けて行う場合はこのセルの実行は不要

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# import chromedriver_binary
# import datetime
# import time
# import re # 正規表現用
# import pandas as pd

# #ブラウザの設定
# chrome_options = webdriver.ChromeOptions()
# # options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# # 拡張機能 Resale Trap を有効にする　staff2@polyphony.tokyo／Muzaiko20230606
# chrome_options.add_extension(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\extensions/resaletrap_3.0.9.crx')
# # Chromeを起動するためのオプションにプロファイルを追加する
# chrome_options.add_argument(r'--user-data-dir=C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\profile')

# #ブラウザの起動する
# browser = webdriver.Chrome(r'C:\Users\amuza\OneDrive\Documents\Python-study\ec_scraping\chromedriver.exe', options=chrome_options)

# # ヤフオク商品ページを開く
# # url = 'https://page.auctions.yahoo.co.jp/jp/auction/v1094580770' $ 固定入力の場合
# print('▼ 対象URLを入力してください') # 標準入力の場合
# url = input()
# browser.get(url)

In [ ]:
# 【動作テスト用】
# ページを直接開く
# browser.get('https://page.auctions.yahoo.co.jp/jp/auction/h1089535961')

In [624]:
# ヤフオク商品名　※ Amazon画面で取得済
yafuoku_name = browser.find_element(By.XPATH, '//h1').text
print('ヤフオク商品名：', yafuoku_name)

ヤフオク商品名： サロニア スピーディーイオンドライヤー SL-013BK ブラック【PSEマークあり】【訳あり※折りまげ部分ゆるみ有】58 00064


In [625]:
# ・出品者が「匿名配送」希望でないこと
# →　商品ページのタイトルに「匿名配送」アイコンがないことで判定
try:
    # 匿名配送だったらその時点で情報取得をやめて画面を閉じる
    privacy = browser.find_element(By.XPATH, '//span[@class="Icon Icon--privacy"]').text # 匿名配送の場合
    print('▲ 匿名配送のため本商品の情報収集を終了します。ヤフオク商品ページのウィンドウハンドラの取得から再実行してください ▲')
    # browser.close()
except:
    privacy = '匿名配送なし'
    print('匿名配送：', privacy)

匿名配送： 匿名配送なし


In [626]:
# アクティブウィンドウ（ヤフオク商品ページ）のURLを取得する
yafuoku_url = browser.current_url
print('ヤフオク商品URL：', yafuoku_url)

ヤフオク商品URL： https://page.auctions.yahoo.co.jp/jp/auction/l1094007746


In [627]:
# ヤフオク現在価格　※ Amazon画面で取得済
yafuoku_price_now = browser.find_element(By.XPATH, '//dd[@class="Price__value"]').text.replace(' ', '').replace(',', '')
print('ヤフオク現在価格：', yafuoku_price_now)

if '税込' in yafuoku_price_now:
    yafuoku_price_now_int = int(yafuoku_price_now.replace('税込', '／').replace('円）', '').split('／')[1])
else:
    yafuoku_price_now_int = int(yafuoku_price_now.split('円')[0])

print('ヤフオク現在価格(int)：', yafuoku_price_now_int)

ヤフオク現在価格： 4000円（税0円）
ヤフオク現在価格(int)： 4000


In [628]:
state = browser.find_element(By.XPATH, '//th[contains(text(), "状態")]/following-sibling::td/a').text
print('状態：', state)

状態： やや傷や汚れあり


In [629]:
bidnum = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[0].text
print('入札数：', bidnum)

入札数： 0件


In [630]:
# ・出品者の評価が97％以上
# →　商品ページの出品者欄の「○○%」で判定
seller_rating = browser.find_element(By.XPATH, '//div[@class="Seller__ratingRatio"]').text
print('評価：', seller_rating)

seller_rating_int = int(seller_rating.split('.')[0].replace('%', ''))
print('評価(int)：', seller_rating_int)

# 評価が 97% 未満だったらその時点で情報取得をやめて画面を閉じる
if seller_rating_int < 97:
    print('▲ 匿名配送のため本商品の情報収集を終了します。次のヤフオク商品ページのウィンドウハンドラの取得から再実行してください ▲')
    # browser.close()

評価： 99.6%
評価(int)： 99


In [631]:
# 評価数は多い方が望ましい
seller_rating_num = browser.find_element(By.XPATH, '//a[@class="Seller__ratingLink"]').text
print('評価数：', seller_rating_num)

評価数： 739


In [632]:
# 送料の「詳細」ポップアップから配送情報を取得する

# 送料の「詳細」をクリックする
# ● 「詳細」ポップアップを開く必要はないのかも
# →　ポップアップを開かないとエラーになる
browser.find_element(By.XPATH, '//a[@id="postageDetailCurrent"]').click()
browser.implicitly_wait(1)
# 配送先から「青森県」を選ぶ
browser.find_element(By.XPATH, '//option[contains(text(), "青森県")]').click()
browser.implicitly_wait(1)

postage_due = browser.find_element(By.XPATH, '//dt[contains(text(), "送料負担")]/following-sibling::dd').text
print('送料負担：', postage_due)

# 配送情報を取得する
try:
    shipping = browser.find_element(By.XPATH, '//dt[@class="BidModal__postageName"]').text
    print('配送方法(詳細)：', shipping)

    postage_1 = browser.find_element(By.XPATH, '//div[@class="BidModal__postagePrice"]').text
    print('送料(詳細)：', postage_1)

    postage_int = int(postage_1.replace('円（税込）', ''))
    print('送料(int)：', postage_int)
except:
    pass

# 「✕」をクリックしてポップアップを閉じる
browser.find_element(By.XPATH, '//div[@class="BidModal__box BidModal__box--postage js-modal-box"]//a[contains(text(), "閉じる")]').click()

postage_from = browser.find_element(By.XPATH, '//p[@class="Price__postageFrom"]').text
print(postage_from)

try:
    send_date = browser.find_element(By.XPATH, '//dt[contains(text(), "発送開始")]/following-sibling::dd').text
except:
    send_date = '情報なし'
finally:
    print('発送開始：', send_date)

try:
    postage_2 = browser.find_element(By.XPATH, '//span[@class="Price__postageValue"]').text
except:
    postage_2 = '情報なし'
finally:
    print('送料(地域込)：', postage_2)

送料負担： 出品者送料無料
配送方法(詳細)： 飛脚宅配便（佐川急便）
送料(詳細)： 無料
発送元：北海道
発送開始： 支払い手続きから3～7日で発送
送料(地域込)： 情報なし


In [633]:
# 商品説明文を取得する
# この内容を基に配送業者情報、営業所止め情報、直接取引情報、着払い情報などを取りたい
yafuoku_info = browser.find_element(By.XPATH, '//div[@class="ProductExplanation__commentArea"]').text
print('商品説明：', yafuoku_info)

商品説明： ◆◆◆　必ずご確認ください　◆◆◆

当店ではネット通販で返品された商品などを主に扱っており、全てアウトレット品扱いとなります。
簡易的なクリーニングのみ行って出品しておりますので、本体・パッケージ・付属品等に記載以外の軽微な汚れや傷みがある場合がございます。


商品詳細
確認内容：
・製品から温風/冷風が出てくる事を確認しております。
・TURBO/SET/COOL/OFFのスイッチ切替が正常にできる事を
確認しております。


[商品状態]
・ハンドルの折りたたみ部分がゆるく、
吹き出し口が下に向てしまうことがあります。
※上記の理由により、訳あり品となります。

・箱に、凹みがあります。


[内容物]
・本体
・ノズル(取り外し可能)
・取扱説明書

※商品画像に写っているものが全てとなります。
※「傷や汚れあり」「全体的に状態が悪い」の商品状態では経年劣化・使用感による傷・汚れがある場合があります。
※プロダクトキー付属商品の場合、使用可否は確認しておりません。キーが使用できないことが理由でのご返金にはご対応致しかねます。

商品仕様
メーカー：サロニア
型番：SL-013BK
梱包サイズ：横幅(約) 14㎝×奥行(約) 14.5㎝×高さ(約) 9㎝

本体素材：PC(ポリカーボネート)
定格電圧：100V
定格消費電力：1200W(TURBO時)
定格周波数：50/60Hz
コード長：約1.7m


------------------------------------------------------------------------

・すべての商品が保障対象外となります。
※付属の保証書については各メーカーにお問い合わせください。

・専門的な質問には、お答え出来ない場合がございます。

・現状での発送となりますので、神経質な方はご入札をご遠慮ください。

・返品はページに表示のない重大な欠陥があり、
商品到着後7日以内にご連絡頂いた場合のみご対応致します。

※お取引の流れはヤフオクヘルプ等をご確認ください。

支払詳細
・かんたん決済
※その他のお支払いにはご対応しておりません。
発送詳細
※沖縄・離島は別途送料がかかります。

・送料無料（沖縄・離島は除く）
ゆうパケット若しくは佐川急便

※沖縄・離島：ゆうパックの正規料

In [634]:
# 【動作テスト用】
# 正規表現での文字列検索

# Pythonで文字列を抽出（位置・文字数、正規表現） 　
# https://note.nkmk.me/python-str-extract/

# 【Python】文字列を検索・抽出する方法【in演算子、findメソッド、正規表現】
# https://python-academia.com/python-str-extract/

# import re

# s = '直接取引限定　直接取り引き限定　直接取引のみ　直接引き取りのみ　要直接引き渡し　直接取引可能　営業所止め限定　営業所止め可能　営業所止め不可　着払い　着払い希望　着払い可能'

# print(re.findall('直接.?.?.?.?.?.?', s))
# print(re.findall('営業所.?.?.?.?', s))
# print(re.findall('着払い.?.?.?.?', s))

# print('直接取引限定：', '直接取引限定' in s)
# print('営業所止め：', '営業所止め' in s)
# print('着払い：', '着払い' in s)

In [635]:
# ・直接引き取り限定でないこと
# →　商品タイトルおよび商品説明文に「直接取引のみ」などの単語がないことで判定
# →　▲ テキストマイニングを活用するとよい？？？
# →　いったん大丈夫そう

# 商品名判定
print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))

# 商品説明判定
print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))

# print('直接取引(商品説明)：', '直接取引' in yafuoku_info)

直接引取(商品名)： []
直接引取(商品説明)： []
手渡し(商品説明) []


In [636]:
# ・配送が着払いでないこと
# →　「送料負担」が「出品者」であること OR 配送情報詳細や商品説明に「着払い」とないこと

print('送料負担：', postage_due) # 上で取得済

# 商品名判定
print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))

# 商品説明判定
print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))

送料負担： 出品者送料無料
着払(商品名)： []
着払(商品説明)： []


In [637]:
# ・配送がヤマト、佐川、日本郵便などであること（直接エンドユーザーまで届けられること、一部運送会社だと営業所留めがあるため）
# ・営業所止めでないこと
# →　商品説明文に「営業所度止め」などの単語がないことで判定。「らくらく家財便」だと安心

# 商品名判定
print('ヤマト(商品名)：', re.findall('.*ヤマト.*', yafuoku_name))

# 商品説明判定
print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*日本郵便.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))

ヤマト(商品名)： []
ヤマト(商品説明)： []
らくらく家財便(商品説明)： []
佐川(商品説明) ['ゆうパケット若しくは佐川急便']
日本郵便(商品説明) []
日本郵便(商品説明) ['ゆうパケット若しくは佐川急便', '※沖縄・離島：ゆうパックの正規料金を頂戴いたします']
営業所止め(商品説明) []


In [638]:
# 訳あり品でないこと（下遠野追加）

# 商品名判定
print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))

# 商品説明判定
print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))

訳あり(商品名)： ['サロニア スピーディーイオンドライヤー SL-013BK ブラック【PSEマークあり】【訳あり※折りまげ部分ゆるみ有】58 00064']
訳あり(商品説明)： ['※上記の理由により、訳あり品となります。']


In [639]:
# オークションの残り日数が一定数あること
# →　商品ページ内の残り時間やオークション終了用低日時で判定

auc_timeleft = browser.find_elements(By.XPATH, '//span[@class="Count__detail"]')[1].text
print('残り日数(画面表示)：', auc_timeleft)

auc_enddatetime = browser.find_element(By.XPATH, '//th[contains(text(), "終了日時")]/following-sibling::td').text
print('終了予定(str型)：', auc_enddatetime)

auc_enddate = auc_enddatetime.split('（')[0]
auc_endtime = auc_enddatetime.split('）')[1]
auc_end = auc_enddate + ' ' + auc_endtime
auc_enddatetime = datetime.datetime.strptime(auc_end, '%Y.%m.%d %H:%M')
print('終了予定(datetime型)：', auc_enddatetime)
# print(type(auc_enddatetime))

dt_now = datetime.datetime.now()
print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))

auc_time_left = auc_enddatetime - dt_now
print('残り時間(詳細計算)：', auc_time_left)

残り日数(画面表示)： 1日
終了予定(str型)： 2023.06.15（木）12:27
終了予定(datetime型)： 2023-06-15 12:27:00
現在時刻： 2023-06-14 02:41:55
残り時間(詳細計算)： 1 day, 9:45:04.767031


In [641]:
# アマゾンでの販売価格が適切に収まること
# →　新品価格を下回っていることで判定
profit = amazon_price - yafuoku_price_now_int - postage_int
print('A-Y現在利益(ヤフオク送料込)：', profit, '円　※Amazon出品手数料は含まれていません')

A-Y現在利益(ヤフオク送料込)： 603 円　※Amazon出品手数料は含まれていません


In [642]:
# ヤフオクとの価格差がある程度あること（利益が取れること）
# →　ResaleTrapで判定するため本プログラムでは対応不要
# →　▲ プログラム上でもできそう
# →　Amazon 出品手数料が分からない
profit_rate = round(profit / yafuoku_price_now_int * 100, 1)
print('現在利益率(ヤフオク送料込)：', profit_rate, '%　※Amazon出品手数料は含まれていません')

現在利益率(ヤフオク送料込)： 15.1 %　※Amazon出品手数料は含まれていません


In [643]:
# 目視確認用の情報一覧表示
print('Amazon商品名：', amazon_name)
# print('Amazon商品URL：', amazon_url)
print(amazon_sold_num)
print('Amazon価格：', amazon_price)
print('ヤフオク商品名：', yafuoku_name)
# print('ヤフオク商品URL：', yafuoku_url)
print('ヤフオク現在価格：', yafuoku_price_now_int)
print('送料(詳細)：', postage_int)
print('送料(地域込)：', postage_2)
print('A-Y現在利益(ヤフオク送料込)：', profit, '※Amazon出品手数料は含まれていません')
print('現在利益率(ヤフオク送料込)：', round(profit_rate, 1), '%　※Amazon出品手数料は含まれていません')
print('状態：', state)
print('入札数：', bidnum)
print('評価：', seller_rating)
print('匿名配送：', privacy)
print('配送方法(詳細)：', shipping)
# print('送料(詳細)：', postage_1)
# print('送料(int)：', postage_int)
print('発送開始：', send_date)
# print('直接引取(商品名)：', re.findall('.*直接.*引.*', yafuoku_name))
print('直接引取(商品説明)：', re.findall('.*直接.*引.*', yafuoku_info))
print('手渡し(商品説明)', re.findall('.*手渡.*', yafuoku_info))
print('送料負担：', postage_due)
# print('着払(商品名)：', re.findall('.*着払.*', yafuoku_name))
print('着払(商品説明)：', re.findall('.*着払.*', yafuoku_info))
print('ヤマト(商品説明)：', re.findall('.*ヤマト.*', yafuoku_info))
print('らくらく家財便(商品説明)：', re.findall('.*らくらく.*', yafuoku_info))
print('佐川(商品説明)', re.findall('.*佐川.*', yafuoku_info))
print('日本郵便(商品説明)', re.findall('.*ゆう.*', yafuoku_info))
print('営業所止め(商品説明)', re.findall('.*止め.*', yafuoku_info))
print('訳あり(商品名)：', re.findall('.*訳.*', yafuoku_name))
print('訳あり(商品説明)：', re.findall('.*訳.*', yafuoku_info))
print('終了予定(datetime型)：', auc_enddatetime)
print('現在時刻：', dt_now.strftime('%Y-%m-%d %H:%M:%S'))
print('残り時間(詳細計算)：', auc_time_left)
print('----- 商品説明 ----------------------------------------------------')
print(yafuoku_info)

Amazon商品名： SALONIA サロニア | スピーディーイオン ドライヤー 【ブラック】 ヘアドライヤー 大風量 速乾 軽量 マイナスイオン 折り畳み式 冷温3段階調整
過去1か月で5000点以上購入されました
Amazon価格： 5373
ヤフオク商品名： サロニア スピーディーイオンドライヤー SL-013BK ブラック【PSEマークあり】【訳あり※折りまげ部分ゆるみ有】58 00064
ヤフオク現在価格： 4000
送料(詳細)： 770
送料(地域込)： 情報なし
A-Y現在利益(ヤフオク送料込)： 603 ※Amazon出品手数料は含まれていません
現在利益率(ヤフオク送料込)： 15.1 %　※Amazon出品手数料は含まれていません
状態： やや傷や汚れあり
入札数： 0件
評価： 99.6%
匿名配送： 匿名配送なし
配送方法(詳細)： 飛脚宅配便（佐川急便）
発送開始： 支払い手続きから3～7日で発送
直接引取(商品説明)： []
手渡し(商品説明) []
送料負担： 出品者送料無料
着払(商品説明)： []
ヤマト(商品説明)： []
らくらく家財便(商品説明)： []
佐川(商品説明) ['ゆうパケット若しくは佐川急便']
日本郵便(商品説明) ['ゆうパケット若しくは佐川急便', '※沖縄・離島：ゆうパックの正規料金を頂戴いたします']
営業所止め(商品説明) []
訳あり(商品名)： ['サロニア スピーディーイオンドライヤー SL-013BK ブラック【PSEマークあり】【訳あり※折りまげ部分ゆるみ有】58 00064']
訳あり(商品説明)： ['※上記の理由により、訳あり品となります。']
終了予定(datetime型)： 2023-06-15 12:27:00
現在時刻： 2023-06-14 02:41:55
残り時間(詳細計算)： 1 day, 9:45:04.767031
----- 商品説明 ----------------------------------------------------
◆◆◆　必ずご確認ください　◆◆◆

当店ではネット通販で返品された商品などを主に扱っており、全てアウトレット品扱いとなります。
簡易的なクリーニングのみ行って出品しておりますので、本体・パッケージ・付属品等に記載以外の軽微

In [644]:
# 出品可否を判定する
judge_flag = 'OK'

# これ以降でNG判定だけを行えば良い。NGになったらフラグを 0 にして、最終的な値で判定する

# 出品者の評価が97％以上
if seller_rating_int >= 97:
    print('◯ 評価')
else:
    print('✕ 評価')
    judge_flag = 'NG'

# 出品者が「匿名配送」希望でないこと
if privacy != '匿名配送':
    print('◯ 匿名配送')
else:
    print('✕ 匿名配送')
    judge_flag = 'NG'
    
# 訳あり品でないこと
if len(re.findall('.*訳.*', yafuoku_name)) == 0 and len(re.findall('.*訳.*', yafuoku_info)) == 0:
    print('◯ 訳あり判定')
else:
    print('✕ 訳あり判定')
    judge_flag = 'NG'

# 直接引き取り限定でないこと
if len(re.findall('.*直接.*引.*', yafuoku_name)) == 0 and len(re.findall('.*直接.*引.*', yafuoku_info)) == 0 and len(re.findall('手渡.{10}', yafuoku_info)) == 0:
    print('◯ 直接引取判定')
else:
    print('✕ 訳あり判定')
    judge_flag = 'NG'

# 配送がヤマト、佐川、日本郵便などであること（直接エンドユーザーまで届けられること、一部運送会社だと営業所留めがあるため）
if len(re.findall('.*止め.*', yafuoku_info)) == 0:
    print('◯ 営業所止め判定')
else:
    print('✕ 営業所止め判定')
    judge_flag = 'NG'
    
# 配送が着払いでないこと
if len(re.findall('着払.*', yafuoku_name)) == 0 and len(re.findall('着払.*', yafuoku_info)) == 0:
    print('◯ 着払い判定')
else:
    print('✕ 着払い判定')
    judge_flag = 'NG'

print('')
print('↑　出品判定：', judge_flag)
print('↓　以下の条件も判定してください')
print('')

# アマゾンでの販売価格が適切に収まること　→　Amazonページで開く時に判定済
# ヤフオクとの価格差がある程度あること（利益が取れること）　→　Amazonページで開く時に判定済
print('Amazon価格：', amazon_price, '円')
print('ヤフオク現在価格：', yafuoku_price_now_int, '円')
print('ヤフオク送料：', postage_int, '円')
print('現在利益(ヤフオク送料込)：', profit, '円　※Amazon出品手数料は含まれていません')
print('現在利益率(ヤフオク送料込)：', round(profit_rate, 1), '%')

# オークションの残り日数が一定数あること
print('残り時間：', round(auc_time_left / datetime.timedelta(days=1), 1), '日')

◯ 評価
◯ 匿名配送
✕ 訳あり判定
◯ 直接引取判定
◯ 営業所止め判定
◯ 着払い判定

↑　出品判定： NG
↓　以下の条件も判定してください

Amazon価格： 5373 円
ヤフオク現在価格： 4000 円
ヤフオク送料： 770 円
現在利益(ヤフオク送料込)： 603 円　※Amazon出品手数料は含まれていません
現在利益率(ヤフオク送料込)： 15.1 %
残り時間： 1.4 日


In [645]:
# 出品（監視対象に追加する）に値するかを判定し、合格の場合は出品する
# とりあえずは目視チェックにしてみる

print('この商品を出品しますか？( YES / NO )')

while True:
    sell_flag = input()

    if sell_flag == 'YES':
        # 「監視対象に追加する」をクリックする。新しいウィンドウでヤフオクの商品ページが開かれる
        browser.find_element(By.XPATH, '//a[contains(text(), "監視対象に追加する")]').click()
        print('出品画面に進みます')
        break
    elif sell_flag == 'NO':
        print('出品を取りやめます。ヤフオク商品ページのウィンドウハンドラの取得から再実行してください')
        # browser.close()
        break
    else:
        print('YES か NO を入力してください')

この商品を出品しますか？( YES / NO )


 YES


出品画面に進みます


In [646]:
# 開いているすべてのウィンドウハンドルを取得（リスト形式）し、新しいウィンドウ（ResaleTrap管理画面）をアクティブにする
handle_array = browser.window_handles
# print(handle_array)
browser.switch_to.window(handle_array[-1])

# ResaleTrapページのウィンドウハンドラの保持　→　なくても大丈夫っぽい
resaletrap_window = browser.current_window_handle
# print(resaletrap_window)
print('ResaleTrap のページをアクティブにしました')

ResaleTrap のページをアクティブにしました


In [647]:
# ■ ここは手入力かなぁ・・・

# 出品情報を設定する
# 予想落札価格
browser.find_elements(By.XPATH, '//input')[0].send_keys(Keys.CONTROL,"a")
browser.find_elements(By.XPATH, '//input')[0].send_keys('0')

# 送料
browser.find_elements(By.XPATH, '//input')[1].send_keys(Keys.CONTROL,"a")
browser.find_elements(By.XPATH, '//input')[1].send_keys(postage_int)
print('送料を設定しました')

# 利益額　※ 利益「率」とのどちらか一方を選択
# browser.find_elements(By.XPATH, '//input')[2].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[2].send_keys('0')

# 利益率　※ 利益「額」とのどちらか一方を選択
# browser.find_elements(By.XPATH, '//input')[3].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[3].send_keys('0')

# リードタイム
# browser.find_elements(By.XPATH, '//input')[4].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[4].send_keys('0')

# 販売価格
# browser.find_elements(By.XPATH, '//input')[5].send_keys(Keys.CONTROL,"a")
# browser.find_elements(By.XPATH, '//input')[5].send_keys('0')

# Sub Condition
# ■ これを選択した後に別の項目に移るとデフォルト値に戻ってしまう
# browser.find_element(By.XPATH, '//option[@value="24"]').click()

# 商品説明　直接入力 OR テンプレート選択
# browser.find_elements(By.XPATH, '//textarea').send_keys(Keys.CONTROL,"a") # エラー「AttributeError: 'list' object has no attribute 'send_keys'」
# browser.find_elements(By.XPATH, '//textarea').send_keys(yafuoku_info)
# print('商品説明を設定しました')

print('金額が「0」の部分、Sub Condition、商品説明を入力してください')

送料を設定しました
金額が「0」の部分、Sub Condition、商品説明を入力してください


In [ ]:
# Amazon 出品手数料を取得する
# amazon_charge = browser.find_elements(By.XPATH, '//★').text
# print(amazon_charge)

## 取得したAmazon情報・ヤフオク情報を蓄積する

In [648]:
# 取得した情報を CSV 出力用にまとめる（出品判定の証跡として残す）
get_datetime = datetime.datetime.now().strftime('%Y/%m/%d %H:%M')

data = {
    '出品判定': judge_flag,
    'Amazon商品名': amazon_name,
    'Amazon価格': amazon_price,
    # 'Amazon出品手数料': amazon_charge,
    'Amazon商品URL': amazon_url,
    'ヤフオク商品名': yafuoku_name,
    'ヤフオク現在価格': yafuoku_price_now,
    'ヤフオク商品URL': yafuoku_url,
    'A-Y現在利益(送料込)': profit,
    '現在利益率(送料込)': profit_rate,
    '情報取得日時': get_datetime,
    }

data_list.append(data)
print('data_list', data_list)

data_list [{'出品': 'OK', 'Amazon商品名': 'SALONIA サロニア | スピーディーイオン ドライヤー 【ブラック】 ヘアドライヤー 大風量 速乾 軽量 マイナスイオン 折り畳み式 冷温3段階調整', 'Amazon価格': 5373, 'Amazon商品URL': 'https://www.amazon.co.jp/SALONIA-%E3%82%B5%E3%83%AD%E3%83%8B%E3%82%A2-SAL19002-%E3%82%B9%E3%83%94%E3%83%BC%E3%83%87%E3%82%A3%E3%83%BC%E3%82%A4%E3%82%AA%E3%83%B3%E3%83%89%E3%83%A9%E3%82%A4%E3%83%A4%E3%83%BC-%E3%83%96%E3%83%A9%E3%83%83%E3%82%AF/dp/B07VCP8JXS/ref=sr_1_1?qid=1686677713&s=kitchen&sr=1-1', 'ヤフオク商品名': 'SALONIA ドライヤー SL-013BK サロニア', 'ヤフオク現在価格': '2800円（税0円）', 'ヤフオク商品URL': 'https://page.auctions.yahoo.co.jp/jp/auction/h1094905192', 'A-Y現在利益(送料込)': 1803, '現在利益率(送料込)': 64.4, '情報取得日時': '2023/06/14 02:40'}, {'出品': 'NG', 'Amazon商品名': 'SALONIA サロニア | スピーディーイオン ドライヤー 【ブラック】 ヘアドライヤー 大風量 速乾 軽量 マイナスイオン 折り畳み式 冷温3段階調整', 'Amazon価格': 5373, 'Amazon商品URL': 'https://www.amazon.co.jp/SALONIA-%E3%82%B5%E3%83%AD%E3%83%8B%E3%82%A2-SAL19002-%E3%82%B9%E3%83%94%E3%83%BC%E3%83%87%E3%82%A3%E3%83%BC%E3%82%A4%E3%82%AA%E3%83%B3%E3%83%89%E3%83%A9%E3%82%A4%E3%

In [650]:
# 「新規出品」ボタンをクリックする
# まずは人力でスタートし、一定の実績が積めてから自動化する
# ■ 「取得したAmazon情報・ヤフオク情報を蓄積する」と1セットにすることで履歴残し漏れ・出品漏れを防げる

browser.find_element(By.XPATH, '//button[contains(text(), "新規出品")]').click()

In [651]:
# ResaleTrap ページを閉じる
browser.close()
print('ResaleTrap のページを閉じました')

ResaleTrap のページを閉じました


In [652]:
# 出品が完了したヤフオク商品ページも閉じる
handle_array = browser.window_handles
# print(handle_array)
browser.switch_to.window(handle_array[-1])

browser.close()
print('出品が完了したヤフオク商品ページを閉じました')
print('次のヤフオク商品ページをアクティブにするところから再実行してください')

# 次のヤフオク商品ページをアクティブにする
# →　「ヤフオクページから情報を取得する」項に戻る

出品が完了したヤフオク商品ページを閉じました
次のヤフオク商品ページをアクティブにするところから再実行してください


## 蓄積したAmazon情報・ヤフオク情報を出力する

In [653]:
# 取得した情報を CSV ファイルに出力する
output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

# 完成版

In [ ]:

def xxx():
    
def yyy():
    
# csvファイルに出力する
def output():
    output_time = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    csv_file_name = 'ヤフオクtoアマゾンデータ_' + output_time + '.csv'
    pd.DataFrame(data_list).to_csv(csv_file_name, encoding='cp932')

def main():
    xxx()
    yyy()
    output()

if __name__ == '__main__':
    main()